# Generating fake tweets
#### Machine learning project - Summer semester 2020

Students: Al Hazwani Ibrahim, Loiacono Diego

The aim of this project is to code a LSTM-layered neural network to feed with text in order to generate some strings with the very same style of writing. In this peculiar situation, we’ve taken into consideration the data from the Twitter Social Network. Our hope in this project is to be able to produce some believable tweets because of its small and not grammatically accurate structure.

In [1]:
import numpy as np
import re
from IPython.display import clear_output
import pandas as pd

In [2]:
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [3]:
def cleandata(filename):
    
    with open(filename, encoding='utf-8-sig') as f:
        text = f.read()

    #tokenize the test linking words to numbers
    if token_type == 'word':
        tokenizer = Tokenizer(char_level = False, filters = '')
    else:
        tokenizer = Tokenizer(char_level = True, filters = '', lower = False)
        
    tokenizer.fit_on_texts([text])

    total_words = len(tokenizer.word_index) + 1

    token_list = tokenizer.texts_to_sequences([text])[0]
    
    return token_list, total_words, tokenizer

In [4]:
def generate_sequences(token_list, step, total_words):
    
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    
    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq

In [5]:
def create_model(step, seq_length, tokenizer, token_list):

    X, y, num_seq = generate_sequences(token_list, step, total_words)
    train_model = True

    X = np.array(X)
    y = np.array(y)

    embedding_size = 100

    text_in = Input(shape = (None,))
    embedding = Embedding(total_words, embedding_size)
    x = embedding(text_in)
    
    x = LSTM(256, return_sequences = True)(x)
    x = Dropout(0.2)(x)
    
    x = LSTM(256, return_sequences = True)(x)
    x = Dropout(0.5)(x)
    
    x = LSTM(128)(x)
    x = Dropout(0.8)(x)
    
    x = Dense(128)(x)
    text_out = Dense(total_words, activation = 'softmax')(x)

    model = Model(text_in, text_out)

    opti = Adam(lr = 0.001) 
    model.compile(loss='categorical_crossentropy', optimizer=opti, metrics=['accuracy']) 

    if train_model:
        epochs = 1000
        batch_size = 32
        model.fit(X, y, epochs=epochs, batch_size=batch_size)
        train_model = False
        
    return model

In [6]:
def sample_with_temp(preds, temperature=0.001):
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [7]:
def generate_text(seed_text, next_words, model, max_sequence_len, temp, tokenizer,all_tokens):
    output_text = seed_text

    start_story = ' | ' * max_sequence_len
    
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = np.random.choice(all_tokens, max_sequence_len)
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = 0.01)
        
        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]
            
        if output_word == "|":
            break
            
        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '  
            
    return output_text

In [ ]:
if  __name__ == "__main__":
    token_type = 'word'
    seq_length = 10
    step = 10

    seed_text = "  "
    gen_words = 400
    temp = 0.01

    filename = "output/clean-tweet.txt"

    token_list, total_words, tokenizer = cleandata(filename)
    X , y , num_seq = generate_sequences(token_list, step, total_words)
    model = create_model(step, seq_length, tokenizer, token_list)
    print(generate_text(seed_text, gen_words, model, seq_length, temp, tokenizer, token_list))

Number of sequences: 726 

Number of sequences: 726 



W0503 14:23:28.780464 4544957888 deprecation_wrapper.py:119] From /Applications/anaconda3/envs/generative/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0503 14:23:28.790848 4544957888 deprecation.py:506] From /Applications/anaconda3/envs/generative/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0503 14:23:29.382117 4544957888 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0503 14:23:29.591388 4544957888 deprecation.py:323] From /Applications/anaconda3/envs/generative/lib/python3.6/site-packages/tensorflow

Epoch 1/1000
726/726 [==============================] - 5s 7ms/step - loss: 7.3595 - acc: 0.0331
Epoch 2/1000
726/726 [==============================] - 2s 3ms/step - loss: 6.3707 - acc: 0.0262
Epoch 3/1000
726/726 [==============================] - 2s 3ms/step - loss: 5.8915 - acc: 0.0331
Epoch 4/1000
726/726 [==============================] - 3s 4ms/step - loss: 5.8059 - acc: 0.0399
Epoch 5/1000
726/726 [==============================] - 3s 4ms/step - loss: 5.7330 - acc: 0.0386
Epoch 6/1000
726/726 [==============================] - 3s 4ms/step - loss: 5.6711 - acc: 0.0248
Epoch 7/1000
726/726 [==============================] - 3s 4ms/step - loss: 5.6812 - acc: 0.0386
Epoch 8/1000
726/726 [==============================] - 3s 4ms/step - loss: 5.7112 - acc: 0.0344
Epoch 9/1000
726/726 [==============================] - 3s 4ms/step - loss: 5.6599 - acc: 0.0386
Epoch 10/1000
726/726 [==============================] - 3s 4ms/step - loss: 5.6379 - acc: 0.0386
Epoch 11/1000
726/726 [======

726/726 [==============================] - 4s 5ms/step - loss: 4.4535 - acc: 0.0620
Epoch 85/1000
726/726 [==============================] - 4s 6ms/step - loss: 4.4179 - acc: 0.0565
Epoch 86/1000
726/726 [==============================] - 5s 6ms/step - loss: 4.4163 - acc: 0.0647
Epoch 87/1000
726/726 [==============================] - 3s 4ms/step - loss: 4.3658 - acc: 0.0551
Epoch 88/1000
726/726 [==============================] - 3s 4ms/step - loss: 4.3288 - acc: 0.0689
Epoch 89/1000
726/726 [==============================] - 3s 5ms/step - loss: 4.2973 - acc: 0.0634
Epoch 90/1000
726/726 [==============================] - 5s 8ms/step - loss: 4.2950 - acc: 0.0675
Epoch 91/1000
726/726 [==============================] - 4s 5ms/step - loss: 4.2599 - acc: 0.0661
Epoch 92/1000
726/726 [==============================] - 4s 5ms/step - loss: 4.2433 - acc: 0.0771
Epoch 93/1000
726/726 [==============================] - 4s 5ms/step - loss: 4.1895 - acc: 0.0923
Epoch 94/1000
726/726 [===========

726/726 [==============================] - 4s 5ms/step - loss: 2.6712 - acc: 0.2369
Epoch 168/1000
726/726 [==============================] - 4s 5ms/step - loss: 2.6400 - acc: 0.2479
Epoch 169/1000
726/726 [==============================] - 4s 5ms/step - loss: 2.7362 - acc: 0.1997
Epoch 170/1000
726/726 [==============================] - 4s 5ms/step - loss: 2.6195 - acc: 0.2369
Epoch 171/1000
726/726 [==============================] - 4s 5ms/step - loss: 2.6293 - acc: 0.2466
Epoch 172/1000
726/726 [==============================] - 3s 4ms/step - loss: 2.5780 - acc: 0.2218
Epoch 173/1000
726/726 [==============================] - 3s 5ms/step - loss: 2.6201 - acc: 0.2507
Epoch 174/1000
726/726 [==============================] - 3s 4ms/step - loss: 2.4902 - acc: 0.2755
Epoch 175/1000
726/726 [==============================] - 3s 4ms/step - loss: 2.4904 - acc: 0.2452
Epoch 176/1000
726/726 [==============================] - 3s 4ms/step - loss: 2.4890 - acc: 0.2534
Epoch 177/1000
726/726 [=

726/726 [==============================] - 3s 3ms/step - loss: 2.1347 - acc: 0.3554
Epoch 250/1000
726/726 [==============================] - 3s 3ms/step - loss: 2.1498 - acc: 0.3471
Epoch 251/1000
726/726 [==============================] - 3s 4ms/step - loss: 2.0003 - acc: 0.4187
Epoch 252/1000
726/726 [==============================] - 4s 5ms/step - loss: 2.0135 - acc: 0.3719
Epoch 253/1000
726/726 [==============================] - 2s 3ms/step - loss: 1.9737 - acc: 0.4118
Epoch 254/1000
726/726 [==============================] - 2s 3ms/step - loss: 2.0112 - acc: 0.3871
Epoch 255/1000
726/726 [==============================] - 2s 3ms/step - loss: 1.9400 - acc: 0.4063
Epoch 256/1000
726/726 [==============================] - 2s 3ms/step - loss: 1.9618 - acc: 0.3967
Epoch 257/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.9111 - acc: 0.4105
Epoch 258/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.9721 - acc: 0.4063
Epoch 259/1000
726/726 [=

726/726 [==============================] - 3s 4ms/step - loss: 1.3565 - acc: 0.5661
Epoch 332/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.3631 - acc: 0.5427
Epoch 333/1000
726/726 [==============================] - 3s 3ms/step - loss: 1.3992 - acc: 0.5482
Epoch 334/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.3646 - acc: 0.5744
Epoch 335/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.4057 - acc: 0.5496
Epoch 336/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.3895 - acc: 0.5882
Epoch 337/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.2589 - acc: 0.5909
Epoch 338/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.3754 - acc: 0.5468
Epoch 339/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.3547 - acc: 0.5565
Epoch 340/1000
726/726 [==============================] - 3s 4ms/step - loss: 1.3153 - acc: 0.5537
Epoch 341/1000
726/726 [=